In [16]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt import expected_returns, risk_models, EfficientFrontier
import subprocess
import json
import matplotlib.pyplot as plt
from datetime import datetime


In [17]:
# --- Import libraries ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("✅ Libraries loaded.")

# --- Load your prepared market data ---
data = pd.read_csv("../data/market_data.csv", index_col=0, parse_dates=True)

# Choose a few tickers for testing (keep it small for now)
assets = ["AAPL_Close", "MSFT_Close", "GOOGL_Close", "AMZN_Close"]
data = data[assets].dropna()

print(f"✅ Data loaded with shape: {data.shape}")
data.tail()


✅ Libraries loaded.
✅ Data loaded with shape: (1468, 4)


,AAPL_Close,MSFT_Close,GOOGL_Close,AMZN_Close
Date,,,,
2025-10-28,269.000000,542.070007,267.470001,229.250000
2025-10-29,269.700012,541.549988,274.570007,230.300003
2025-10-30,271.399994,525.760010,281.480011,222.860001
2025-10-31,270.369995,517.809998,281.190002,244.220001
2025-11-03,269.049988,517.030029,283.720001,254.000000


In [18]:

class BaseAgent:
    def perceive(self, data):
        raise NotImplementedError

    def train(self):
        raise NotImplementedError

    def act(self):
        raise NotImplementedError


## Conservative Agent

In [19]:
class ConservativeAgent(BaseAgent):
    def __init__(self, lambdas=[5, 10, 20]):
        self.lambdas = lambdas
        self.best_lambda = None
        self.returns = None
        self.last_weights = None

    def perceive(self, data):
        self.returns = np.log(data / data.shift(1)).dropna()

    def train(self):
        best_sr, best_lam = -np.inf, None
        for lam in self.lambdas:
            mu = self.returns.mean()
            cov = self.returns.cov()
            inv_cov = np.linalg.pinv(cov)
            w = inv_cov @ mu / lam
            w /= w.sum()
            port = (self.returns @ w).dropna()
            sr = port.mean() / port.std()
            if sr > best_sr:
                best_sr, best_lam = sr, lam
        self.best_lambda = best_lam

    def act(self):
        mu = self.returns.mean()
        cov = self.returns.cov()
        inv_cov = np.linalg.pinv(cov)
        lam = self.best_lambda or 10
        w = inv_cov @ mu / lam
        w /= w.sum()
        self.last_weights = pd.Series(w, index=self.returns.columns, name="Conservative")
        return self.last_weights

    def speak(self):
        if self.last_weights is None:
            return "I haven’t acted yet."
        top_assets = self.last_weights.nlargest(2).index.tolist()
        return f"As a conservative agent, I favor {top_assets} due to their stable returns and low volatility."


## Risk Loving Agent

In [20]:
class RiskLovingAgent(BaseAgent):
    def __init__(self, lookbacks=[20, 60]):
        self.lookbacks = lookbacks
        self.best_lb = None
        self.returns = None
        self.last_weights = None

    def perceive(self, data):
        self.returns = np.log(data / data.shift(1)).dropna()

    def train(self):
        best_sr, best_lb = -np.inf, None
        for lb in self.lookbacks:
            momentum = self.returns.rolling(lb).mean().iloc[-1]
            weights = momentum.clip(lower=0)
            weights /= weights.sum()
            port = (self.returns @ weights).dropna()
            sr = port.mean() / port.std()
            if sr > best_sr:
                best_sr, best_lb = sr, lb
        self.best_lb = best_lb

    def act(self):
        momentum = self.returns.rolling(self.best_lb).mean().iloc[-1]
        weights = momentum.clip(lower=0)
        weights /= weights.sum()
        self.last_weights = pd.Series(weights, index=self.returns.columns, name="RiskLoving")
        return self.last_weights

    def speak(self):
        if self.last_weights is None:
            return "I haven’t acted yet."
        top_assets = self.last_weights.nlargest(2).index.tolist()
        return f"As a risk-loving agent, I prefer {top_assets} — they have the strongest upward momentum."


## Rational Agent

In [21]:
class RationalAgent(BaseAgent):
    def __init__(self, vol_window=60):
        self.vol_window = vol_window
        self.returns = None
        self.last_weights = None

    def perceive(self, data):
        self.returns = np.log(data / data.shift(1)).dropna()

    def train(self):
        pass  # no training needed

    def act(self):
        vol = self.returns.rolling(self.vol_window).std().iloc[-1]
        inv_vol = 1 / vol
        weights = inv_vol / inv_vol.sum()
        self.last_weights = pd.Series(weights, index=self.returns.columns, name="Rational")
        return self.last_weights

    def speak(self):
        if self.last_weights is None:
            return "I haven’t acted yet."
        top_assets = self.last_weights.nlargest(2).index.tolist()
        return f"As a rational agent, I emphasize diversification but slightly favor {top_assets} because of moderate volatility and solid returns."


## sentiment Agent

In [22]:
import numpy as np
import pandas as pd
import random

class SentimentAgent:
    """
    The SentimentAgent observes market mood (real or simulated)
    and assigns a sentiment score (-1 to +1) to each asset.
    This version works offline (random simulation),
    but can be extended later to use real data (FinBERT, news API, etc.).
    """
    def __init__(self, assets, mode="simulate"):
        self.assets = assets
        self.mode = mode
        self.sentiment = pd.Series(index=assets, dtype=float)
        self.explanations = {}

    def perceive(self, text_data=None):
        """
        Collects and processes sentiment signals.
        If mode='simulate', generate synthetic sentiment.
        Otherwise, parse text_data or API output.
        """
        if self.mode == "simulate":
            # Generate random sentiment values between -1 (bearish) and +1 (bullish)
            self.sentiment = pd.Series(
                np.random.uniform(-1, 1, len(self.assets)),
                index=self.assets,
                name="Sentiment"
            )
            # Generate short textual explanations
            self.explanations = {
                asset: (
                    "bullish" if s > 0.3 else
                    "neutral" if abs(s) <= 0.3 else
                    "bearish"
                )
                for asset, s in self.sentiment.items()
            }
        else:
            # You can later plug in FinBERT, news API, or Twitter API here
            raise NotImplementedError("Only 'simulate' mode implemented for now.")

        print("📰 SentimentAgent updated market sentiment.")

    def speak(self):
        """
        Summarizes overall sentiment in natural language.
        """
        pos = (self.sentiment > 0.3).sum()
        neg = (self.sentiment < -0.3).sum()
        neutral = len(self.assets) - pos - neg

        msg = (
            f"Market sentiment shows {pos} bullish, "
            f"{neg} bearish, and {neutral} neutral assets. "
            f"Average sentiment = {self.sentiment.mean():.2f}."
        )
        return msg

    def act(self):
        """
        Converts sentiment into pseudo-portfolio weights (more positive = higher weight).
        """
        weights = (self.sentiment - self.sentiment.min())  # shift to positive
        if weights.sum() > 0:
            weights /= weights.sum()
        else:
            weights[:] = 1 / len(weights)

        print("📈 Sentiment-based preference weights generated.")
        return weights


## Aggreagator

In [23]:
class OllamaAggregator:
    def __init__(self, model="llama3", risk_aversion=3):
        self.model = model
        self.risk_aversion = risk_aversion

    def act(self, agent_decisions, sentiment, agent_messages):
        """
        Combine all agents' weights and messages into a single LLM reasoning prompt.
        """
        # Build full conversation
        reasoning = "\n\n".join([
            f"{name} says: {msg}" for name, msg in agent_messages.items()
        ])

        sentiment_msg = ", ".join([
            f"{k}: {'Positive' if v>0 else 'Negative'} ({v:.2f})"
            for k, v in sentiment.items()
        ])

        prompt = f"""
        You are a senior investment strategist moderating a discussion among 3 portfolio agents.
        The agents have just shared their reasoning:

        {reasoning}

        Market sentiment:
        {sentiment_msg}

        Investor risk aversion: {self.risk_aversion}.

        Please summarize their arguments, highlight key disagreements,
        and produce a balanced, reasoned final portfolio.
        Return **only** JSON in the format:
        {{
            "AAPL": 0.3,
            "MSFT": 0.4,
            "GOOGL": 0.3
        }}
        """

        import ollama, json
        print("🧠 Ollama aggregator facilitating discussion...")
        response = ollama.chat(model=self.model, messages=[{"role": "user", "content": prompt}])

        text = response["message"]["content"]
        print("\n🤖 Ollama reasoning summary:\n", text)

        try:
            json_start = text.find("{")
            json_end = text.rfind("}") + 1
            json_text = text[json_start:json_end]
            weights_dict = json.loads(json_text)
            final_weights = pd.Series(weights_dict)
            final_weights /= final_weights.sum()
        except Exception as e:
            print(f"⚠️ Could not parse JSON ({e}) — fallback to averaging.")
            df = pd.concat(agent_decisions.values(), axis=1)
            final_weights = df.mean(axis=1)
            final_weights /= final_weights.sum()

        return final_weights


## Communication Loop

In [26]:
# --- Initialize ---
agents = {
    "Conservative": ConservativeAgent(),
    "RiskLoving": RiskLovingAgent(),
    "Rational": RationalAgent()
}

sentiment_agent = SentimentAgent(data.columns)
llm_agg = OllamaAggregator(model="llama3")

# --- Step 1: Perception & Training ---
for name, agent in agents.items():
    agent.perceive(data)
    agent.train()

# SentimentAgent perceives the market
sentiment_agent.perceive()

# --- Step 2: Each agent acts & speaks ---
decisions, messages = {}, {}
for name, agent in agents.items():
    w = agent.act()
    msg = agent.speak()
    decisions[name] = w
    messages[name] = msg
    print(f"{name} says: {msg}\n")

# Add sentiment agent’s qualitative summary
sentiment_msg = sentiment_agent.speak()
messages["SentimentAgent"] = sentiment_msg
print(f"SentimentAgent says: {sentiment_msg}\n")

# --- Step 3: LLM Aggregator merges reasoning ---
final_weights = llm_agg.act(
    agent_decisions=decisions,
    sentiment=sentiment_agent.sentiment,
    agent_messages=messages
)

print("\n✅ Final Aggregated Portfolio:")
display(final_weights.sort_values(ascending=False))


📰 SentimentAgent updated market sentiment.
Conservative says: As a conservative agent, I favor ['GOOGL_Close', 'AAPL_Close'] due to their stable returns and low volatility.

RiskLoving says: As a risk-loving agent, I prefer ['GOOGL_Close', 'AAPL_Close'] — they have the strongest upward momentum.

Rational says: As a rational agent, I emphasize diversification but slightly favor ['MSFT_Close', 'AAPL_Close'] because of moderate volatility and solid returns.

SentimentAgent says: Market sentiment shows 0 bullish, 2 bearish, and 2 neutral assets. Average sentiment = -0.38.

🧠 Ollama aggregator facilitating discussion...

🤖 Ollama reasoning summary:
 Summary of arguments:

* Conservative and RiskLoving agents favor ['GOOGL_Close', 'AAPL_Close'] due to their stable returns and low volatility, as well as strong upward momentum.
* Rational agent emphasizes diversification and slightly favors ['MSFT_Close', 'AAPL_Close'] due to moderate volatility and solid returns.
* SentimentAgent highlights 

MSFT     0.4
AAPL     0.3
GOOGL    0.3
dtype: float64

In [27]:
# --- Initialize ---
agents = {
    "Conservative": ConservativeAgent(),
    "RiskLoving": RiskLovingAgent(),
    "Rational": RationalAgent()
}

sentiment_agent = SentimentAgent(data.columns)
llm_agg = OllamaAggregator(model="llama3")

# --- Step 1: Perception & Training ---
for name, agent in agents.items():
    agent.perceive(data)
    agent.train()

# SentimentAgent perceives the market
sentiment_agent.perceive()

# --- Step 2: Each agent acts & speaks ---
decisions, messages = {}, {}
for name, agent in agents.items():
    w = agent.act()
    msg = agent.speak()
    decisions[name] = w
    messages[name] = msg
    print(f"{name} says: {msg}\n")

# Add sentiment agent’s qualitative summary
sentiment_msg = sentiment_agent.speak()
messages["SentimentAgent"] = sentiment_msg
print(f"SentimentAgent says: {sentiment_msg}\n")

# --- Step 3: LLM Aggregator merges reasoning ---
final_weights = llm_agg.act(
    agent_decisions=decisions,
    sentiment=sentiment_agent.sentiment,
    agent_messages=messages
)

print("\n✅ Final Aggregated Portfolio:")
display(final_weights.sort_values(ascending=False))


📰 SentimentAgent updated market sentiment.
Conservative says: As a conservative agent, I favor ['GOOGL_Close', 'AAPL_Close'] due to their stable returns and low volatility.

RiskLoving says: As a risk-loving agent, I prefer ['GOOGL_Close', 'AAPL_Close'] — they have the strongest upward momentum.

Rational says: As a rational agent, I emphasize diversification but slightly favor ['MSFT_Close', 'AAPL_Close'] because of moderate volatility and solid returns.

SentimentAgent says: Market sentiment shows 0 bullish, 3 bearish, and 1 neutral assets. Average sentiment = -0.45.

🧠 Ollama aggregator facilitating discussion...

🤖 Ollama reasoning summary:
 Summary of arguments:

* Conservative and RiskLoving agents favor ['GOOGL_Close', 'AAPL_Close'] for their stable returns and low volatility, respectively.
* Rational agent emphasizes diversification and slightly favors ['MSFT_Close', 'AAPL_Close'] due to moderate volatility and solid returns.
* SentimentAgent highlights the negative sentiment a

MSFT     0.4
AAPL     0.3
GOOGL    0.3
dtype: float64